### Import Functions

In [1]:
from ipynb.fs.full.utilities import *
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
import pickle
from sklearn.metrics import classification_report

### Data Prep

In [2]:
X_train_60 = load_json('../../data/Train_60_X.json')
y_train_60 = load_json('../../data/Train_60_y.json')
X_dev = load_json('../../data/Dev_X.json')
y_dev = load_json('../../data/Dev_y.json')

In [3]:
verify_prepped_data(X_train_60, y_train_60)

In [4]:
verify_prepped_data(X_dev, y_dev)

In [5]:
X_train_60_flat = [word for sentence in X_train_60 for word in sentence]
y_train_60_flat = [label for sentence in y_train_60 for label in sentence]
X_dev_flat = [word for sentence in X_dev for word in sentence]
y_dev_flat = [label for sentence in y_dev for label in sentence]

In [6]:
len(X_train_60_flat) == len(y_train_60_flat)

True

In [7]:
len(X_dev_flat) == len(y_dev_flat)

True

In [8]:
X_train_60[0]

[{'word': '^',
  'sent_len': 6,
  'upos': 'PART',
  'first_word_in_sent': 'нека',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Te',
  'BOS': True,
  'next_word': 'нека',
  'next_word_upos': 'PART',
  'next_word_xpos_type': 'Tv',
  'word_after_next_word': 'не',
  'word_after_next_word_upos': 'PART',
  'word_after_next_word_xpos_type': 'Tn',
  'contains_publicistic_word': False},
 {'word': 'нека',
  'sent_len': 6,
  'upos': 'PART',
  'first_word_in_sent': 'нека',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Tv',
  'prev_word': '^',
  'prev_word_upos': 'PART',
  'prev_word_xpos_type': 'Te',
  'next_word': 'не',
  'next_word_upos': 'PART',
  'next_word_x

In [9]:
X_dev[0]

[{'word': '^',
  'sent_len': 42,
  'upos': 'CCONJ',
  'first_word_in_sent': 'но',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'punct',
  'BOS': True,
  'next_word': 'но',
  'next_word_upos': 'CCONJ',
  'next_word_xpos_type': 'Cc',
  'word_after_next_word': 'синовете',
  'word_after_next_word_upos': 'NOUN',
  'word_after_next_word_xpos_type': 'Nc',
  'word_after_next_word_xpos_gender_number_article': 'mpd',
  'contains_publicistic_word': True},
 {'word': 'но',
  'sent_len': 42,
  'upos': 'CCONJ',
  'first_word_in_sent': 'но',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Cc',
  'prev_word': '^',
  'prev_word_upos': 'CCONJ',
  'prev_word_xpos_type': 'pu

In [10]:
v = DictVectorizer()
X_train_60_svm = v.fit_transform(X_train_60_flat)
X_dev_svm = v.transform(X_dev_flat)

In [11]:
le = preprocessing.LabelEncoder()
y_train_60_svm = le.fit_transform(y_train_60_flat)
y_dev_svm = le.transform(y_dev_flat)

In [12]:
le.classes_

array(['', '!', '"', '",', '".', '(', ')', '),', ').', ',', '-', '.',
       '...', ':', ':"', ';', '?'], dtype='<U3')

### SVM Model

In [14]:
%%time
clf = LinearSVC()
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 2h 49min 18s, sys: 10.5 s, total: 2h 49min 28s
Wall time: 2h 49min 34s


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [17]:
pkl_filename = '../../data/svm_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [23]:
labels = list(clf.classes_)

In [24]:
y_pred = clf.predict(X_dev_svm)

In [51]:
target_names = list(le.classes_)

In [44]:
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.969     0.975     0.972    240418
           !      0.477     0.279     0.352       373
           "      0.493     0.242     0.324      2243
          ",      0.686     0.591     0.635       848
          ".      0.282     0.197     0.232       300
           (      0.882     0.913     0.897      1693
           )      0.856     0.854     0.855      1003
          ),      0.768     0.670     0.716       385
          ).      0.908     0.945     0.926       271
           ,      0.694     0.727     0.710     18651
           -      0.332     0.135     0.192       935
           .      0.951     0.968     0.959     11435
         ...      0.200     0.043     0.071        70
           :      0.692     0.689     0.691       935
          :"      0.538     0.103     0.173        68
           ;      0.150     0.073     0.098       628
           ?      0.747     0.859     0.799       453

    accuracy              

In [ ]:
target_names.remove('')

In [ ]:
labels.remove(0)

In [54]:
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

           !      0.477     0.279     0.352       373
           "      0.493     0.242     0.324      2243
          ",      0.686     0.591     0.635       848
          ".      0.282     0.197     0.232       300
           (      0.882     0.913     0.897      1693
           )      0.856     0.854     0.855      1003
          ),      0.768     0.670     0.716       385
          ).      0.908     0.945     0.926       271
           ,      0.694     0.727     0.710     18651
           -      0.332     0.135     0.192       935
           .      0.951     0.968     0.959     11435
         ...      0.200     0.043     0.071        70
           :      0.692     0.689     0.691       935
          :"      0.538     0.103     0.173        68
           ;      0.150     0.073     0.098       628
           ?      0.747     0.859     0.799       453

   micro avg      0.769     0.743     0.756     40291
   macro avg      0.604   

In [55]:
actual_y_pred = le.inverse_transform(y_pred)

In [ ]:
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)

In [64]:
print(punctuated_sentences[:10679])

но синовете им не умъртви, защото постъпи според писаното в закона, в книгата на мойсей, където господ заповяда и каза: бащите, да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря, ще отворя устата ти и ти им кажи: така казва господ, бог, който слуша, нека слуша, а който не слуша, нека не слуша, защото са бунтовен дом. защото кой е по-голям, този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели, както в предишните дни, заявява господ. а той беше на задната част заспал на възглавница. ефрем е като птица славата му ще отлети. и така той си отиде от него на известно разстояние. не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени. а когато ахитофел видя, че съветът му не се изпълни оседла магарето си и стана и отиде у дома си в своя град и нареди домашните си, работи и се обеси. ако умре някое животно от добитъка, който можете да ядете, който се допре до 

#### Results: Better recall and worse precision compared to CRF's equivalent

### SVM - scaled data, dual=False, balanced classed_weight, incresed C and lower max_iter

In [16]:
%%time
clf = make_pipeline(StandardScaler(with_mean=False), LinearSVC(dual=False, class_weight='balanced', C=1.5, max_iter=200))
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 1h 2min 34s, sys: 6min 41s, total: 1h 9min 16s
Wall time: 12min 27s


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('linearsvc',
                 LinearSVC(C=1.5, class_weight='balanced', dual=False,
                           max_iter=200))])

In [17]:
pkl_filename = '../../data/svm_v2_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [18]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.970     0.948     0.959    240418
           !      0.245     0.279     0.261       373
           "      0.271     0.401     0.323      2243
          ",      0.486     0.554     0.518       848
          ".      0.130     0.153     0.141       300
           (      0.885     0.894     0.889      1693
           )      0.669     0.821     0.737      1003
          ),      0.563     0.673     0.613       385
          ).      0.612     0.900     0.728       271
           ,      0.625     0.669     0.647     18651
           -      0.100     0.173     0.127       935
           .      0.931     0.951     0.941     11435
         ...      0.038     0.071     0.049        70
           :      0.486     0.718     0.579       935
          :"      0.070     0.176     0.100        68
           ;      0.083     0.188     0.115       628
           ?      0.568     0.603     0.585       453

    accuracy              

#### Result: Maybe the iterations were too little

### SVM - scaled data, dual=False, balanced classed_weight, incresed C

In [19]:
%%time
clf = make_pipeline(StandardScaler(with_mean=False), LinearSVC(dual=False, class_weight='balanced', C=1.5))
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 4h 12min 53s, sys: 28min 38s, total: 4h 41min 32s
Wall time: 50min 20s


Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('linearsvc',
                 LinearSVC(C=1.5, class_weight='balanced', dual=False))])

In [20]:
pkl_filename = '../../data/svm_v3_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [21]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.969     0.949     0.959    240418
           !      0.235     0.284     0.257       373
           "      0.270     0.396     0.321      2243
          ",      0.502     0.540     0.520       848
          ".      0.121     0.140     0.130       300
           (      0.886     0.893     0.890      1693
           )      0.705     0.809     0.753      1003
          ),      0.581     0.673     0.623       385
          ).      0.635     0.893     0.742       271
           ,      0.620     0.666     0.643     18651
           -      0.101     0.170     0.127       935
           .      0.928     0.946     0.937     11435
         ...      0.038     0.057     0.046        70
           :      0.485     0.705     0.574       935
          :"      0.079     0.176     0.109        68
           ;      0.093     0.180     0.123       628
           ?      0.583     0.598     0.590       453

    accuracy              

In [24]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10735])

но синовете им не умъртви, защото постъпи според писаното в закона, в книгата на мойсей, където господ заповяда и каза: бащите, да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря- ще отворя устата ти и ти им кажи: така казва господ: бог, който слуша, нека слуша; а който не слуша; нека не слуша; защото са бунтовен дом. защото:" кой е по-голям, този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели, както в предишните дни, заявява господ. а той беше на задната част; заспал на възглавница. ефрем е като птица" славата му ще отлети. и така той си отиде от него на известно разстояние! не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени. а когато ахитофел видя, че съветът му не се изпълни оседла магарето си, и стана и отиде у дома си в своя град и нареди домашните си, работи; и се обеси. ако умре някое животно от добитъка, който можете да ядете, който се доп

#### No great difference compared to max_iter=200

### SVM - scaled data, dual=False, balanced classed_weight

In [26]:
%%time
clf = make_pipeline(StandardScaler(with_mean=False), LinearSVC(dual=False, class_weight='balanced'))
clf.fit(X_train_60_svm, y_train_60_svm)

UsageError: Line magic function `%%time` not found.


In [28]:
pkl_filename = '../../data/svm_v4_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [29]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.969     0.949     0.959    240418
           !      0.232     0.279     0.253       373
           "      0.270     0.396     0.321      2243
          ",      0.495     0.541     0.517       848
          ".      0.122     0.140     0.130       300
           (      0.886     0.893     0.890      1693
           )      0.707     0.808     0.754      1003
          ),      0.578     0.673     0.622       385
          ).      0.635     0.893     0.742       271
           ,      0.621     0.665     0.642     18651
           -      0.101     0.170     0.127       935
           .      0.928     0.947     0.937     11435
         ...      0.038     0.057     0.046        70
           :      0.485     0.705     0.575       935
          :"      0.078     0.176     0.108        68
           ;      0.091     0.174     0.119       628
           ?      0.586     0.600     0.593       453

    accuracy              

#### Result: There was a better performance with a higher C

### SVM - scaled data, dual=False and C=3

In [30]:
%%time
clf = make_pipeline(StandardScaler(with_mean=False), LinearSVC(dual=False, C=3))
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 4h 9min 24s, sys: 29min 6s, total: 4h 38min 30s
Wall time: 49min 23s


Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('linearsvc', LinearSVC(C=3, dual=False))])

In [31]:
pkl_filename = '../../data/svm_v5_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [32]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.962     0.969     0.966    240418
           !      0.247     0.236     0.241       373
           "      0.319     0.251     0.281      2243
          ",      0.502     0.501     0.501       848
          ".      0.088     0.117     0.100       300
           (      0.898     0.888     0.893      1693
           )      0.747     0.769     0.758      1003
          ),      0.484     0.657     0.557       385
          ).      0.739     0.908     0.815       271
           ,      0.709     0.620     0.662     18651
           -      0.113     0.142     0.126       935
           .      0.934     0.946     0.940     11435
         ...      0.051     0.057     0.054        70
           :      0.517     0.634     0.570       935
          :"      0.103     0.088     0.095        68
           ;      0.072     0.080     0.076       628
           ?      0.546     0.678     0.605       453

    accuracy              

#### Results: Drop in recall but greater precision

In [34]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10671])

но синовете им не умъртви, защото постъпи според писаното в закона в книгата на мойсей, където господ заповяда и каза бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря- ще отворя устата ти и ти им кажи: така казва господ, бог, който слуша, нека слуша; а който не слуша, нека не слуша, защото са бунтовен дом. защото кой е по-голям, този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели, както в предишните дни, заявява господ. а той беше на задната част; заспал на възглавница. ефрем е като птица славата му ще отлети. и така той си отиде от него на известно разстояние! не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени. а когато ахитофел видя, че съветът му не се изпълни оседла магарето си, и стана и отиде у дома си в своя град и нареди домашните си, работи; и се обеси. ако умре някое животно от добитъка, който можете да ядете, който се допре до 

### SVM - scaled data, dual=False and C=3, class_weight='balanced'

In [13]:
%%time
clf = make_pipeline(StandardScaler(with_mean=False), LinearSVC(dual=False, C=3, class_weight='balanced'))
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 4h 10min 36s, sys: 31min 22s, total: 4h 41min 58s
Wall time: 52min 28s


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('linearsvc',
                 LinearSVC(C=3, class_weight='balanced', dual=False))])

In [14]:
pkl_filename = '../../data/svm_v6_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [15]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.969     0.949     0.959    240418
           !      0.233     0.279     0.254       373
           "      0.269     0.396     0.320      2243
          ",      0.498     0.546     0.521       848
          ".      0.121     0.140     0.130       300
           (      0.888     0.892     0.890      1693
           )      0.699     0.810     0.750      1003
          ),      0.581     0.673     0.623       385
          ).      0.634     0.893     0.741       271
           ,      0.621     0.666     0.643     18651
           -      0.101     0.170     0.127       935
           .      0.929     0.947     0.938     11435
         ...      0.038     0.057     0.045        70
           :      0.486     0.706     0.576       935
          :"      0.079     0.176     0.110        68
           ;      0.093     0.182     0.123       628
           ?      0.586     0.607     0.597       453

    accuracy              

#### Worse results with balanced weight

### SVM - no scaling, dual=False, C=3

In [16]:
%%time
clf = LinearSVC(dual=False, C=3)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 2h 16min 45s, sys: 16min 30s, total: 2h 33min 15s
Wall time: 27min 48s


LinearSVC(C=3, dual=False)

In [17]:
pkl_filename = '../../data/svm_v7_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [18]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.963     0.985     0.974    240418
           !      0.482     0.247     0.326       373
           "      0.514     0.218     0.306      2243
          ",      0.693     0.553     0.615       848
          ".      0.344     0.150     0.209       300
           (      0.958     0.905     0.931      1693
           )      0.878     0.831     0.853      1003
          ),      0.777     0.688     0.730       385
          ).      0.937     0.930     0.933       271
           ,      0.769     0.656     0.708     18651
           -      0.327     0.136     0.192       935
           .      0.946     0.977     0.961     11435
         ...      0.214     0.043     0.071        70
           :      0.814     0.646     0.720       935
          :"      0.727     0.118     0.203        68
           ;      0.214     0.083     0.119       628
           ?      0.810     0.779     0.794       453

    accuracy              

In [19]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10671])

но синовете им не умъртви, защото постъпи според писаното в закона в книгата на мойсей, където господ заповяда и каза: бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря ще отворя устата ти и ти им кажи: така казва господ бог, който слуша, нека слуша; а който не слуша, нека не слуша, защото са бунтовен дом. защото кой е по-голям, този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели, както в предишните дни, заявява господ. а той беше на задната част заспал на възглавница. ефрем е като птица славата му ще отлети. и така той си отиде от него на известно разстояние. не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени а когато ахитофел видя, че съветът му не се изпълни оседла магарето си и стана и отиде у дома си в своя град и нареди домашните си, работи и се обеси. ако умре някое животно от добитъка, който можете да ядете, който се допре до мърша

#### Result: Better performance without scaling.

In [20]:
%%time
clf = LinearSVC(dual=False, C=4)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 2h 16min 17s, sys: 16min 42s, total: 2h 33min
Wall time: 27min 49s


LinearSVC(C=4, dual=False)

In [21]:
pkl_filename = '../../data/svm_v8_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [22]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.963     0.985     0.974    240418
           !      0.470     0.252     0.328       373
           "      0.498     0.221     0.306      2243
          ",      0.688     0.553     0.613       848
          ".      0.326     0.153     0.209       300
           (      0.961     0.905     0.932      1693
           )      0.876     0.833     0.854      1003
          ),      0.776     0.694     0.733       385
          ).      0.937     0.937     0.937       271
           ,      0.773     0.654     0.708     18651
           -      0.305     0.136     0.188       935
           .      0.946     0.975     0.961     11435
         ...      0.214     0.043     0.071        70
           :      0.814     0.645     0.720       935
          :"      0.615     0.118     0.198        68
           ;      0.192     0.084     0.117       628
           ?      0.803     0.784     0.793       453

    accuracy              

#### Result: No improvement

In [23]:
%%time
clf = LinearSVC(C=3)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 3h 4min 19s, sys: 21.7 s, total: 3h 4min 41s
Wall time: 3h 5min 11s


LinearSVC(C=3)

In [24]:
pkl_filename = '../../data/svm_v9_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [25]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.964     0.984     0.974    240418
           !      0.445     0.282     0.345       373
           "      0.510     0.222     0.309      2243
          ",      0.725     0.531     0.613       848
          ".      0.350     0.143     0.203       300
           (      0.958     0.904     0.930      1693
           )      0.874     0.846     0.860      1003
          ),      0.790     0.673     0.727       385
          ).      0.931     0.941     0.936       271
           ,      0.750     0.680     0.713     18651
           -      0.426     0.120     0.187       935
           .      0.947     0.975     0.961     11435
         ...      0.222     0.057     0.091        70
           :      0.836     0.636     0.723       935
          :"      0.727     0.118     0.203        68
           ;      0.264     0.038     0.067       628
           ?      0.793     0.788     0.791       453

    accuracy              

In [26]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10671])

но синовете им не умъртви, защото постъпи според писаното в закона, в книгата на мойсей, където господ заповяда и каза: бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря, ще отворя устата ти и ти им кажи: така казва господ, бог, който слуша, нека слуша, а който не слуша, нека не слуша, защото са бунтовен дом. защото кой е по-голям този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели, както в предишните дни, заявява господ. а той беше на задната част заспал на възглавница. ефрем е като птица славата му ще отлети. и така той си отиде от него на известно разстояние. не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени. а когато ахитофел видя, че съветът му не се изпълни оседла магарето си и стана и отиде у дома си в своя град и нареди домашните си, работи и се обеси. ако умре някое животно от добитъка, който можете да ядете, който се допре до мъ

#### Result: Gets a little bit higher recall and a little bit lower precision - a little bit better F-score. Not that very different.